# Data visualization : milestone 1 

## Dataset

For our project, we found interesting to take a record of some of the 911 calls in Montgomery County, in the Commonwealth of Pennsylvania.

You can find the data set here : https://www.kaggle.com/datasets/mchirico/montcoalert.

#### Read the data

In [ ]:

import datetime
import json
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
from urllib.request import urlopen
import warnings

warnings.filterwarnings("ignore")

sns.set(style="white", color_codes=True)

In [ ]:
FILE_LOCATION = {"calls": "./input/archive.zip", "revenues":"./input/montgomery_county_revenues.csv"}

In [ ]:
dateparse = lambda x: datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S')

calls_data = pd.read_csv(
    filepath_or_buffer = FILE_LOCATION["calls"],
    header = 0,
    names = ['lat', 'lng', 'desc', 'zip', 'title', 'timeStamp', 'twp', 'addr', 'e'],
    dtype = {
        'lat':str,'lng':str,'desc':str, 'zip':str,
        'title':str, 'timeStamp':str, 'twp':str, 'addr':str, 'e':int
        }, 
    parse_dates = ['timeStamp'],
    date_parser = dateparse)

calls_data.timeStamp = pd.DatetimeIndex(calls_data.timeStamp)

In [ ]:
calls_data.head()

From a first look, the data set seems realtively clean. There could be some NaN on the zip information or other columns.

In [ ]:
revenues_data = pd.read_csv(
    filepath_or_buffer = FILE_LOCATION["revenues"],
    header = 0,
    names = ['zipcode', 'med_revenue'],
    dtype = {'zipcode': str, 'med_revenue': "Int64"},
    sep = ";",
    na_values = "Nothing"
    )

In [ ]:
revenues_data.head()

#### First look at the calls data

In [ ]:
calls_data.info()

Expect for the zip column, all the others columns do not contain any missing values.

The description column must be pretty unique. Let us check if there is any duplicates on that column.

In [ ]:
len(calls_data['desc'].drop_duplicates())

In [ ]:
print("percentage of duplicates dropped in the description column :", (1 - len(calls_data['desc'].drop_duplicates()) / len(calls_data))*100)

This column does not seem to have loads (less than 1%) of duplicates values. It is important since it contains lots of valuable information.

The data set seems relatively big enough to perfom some interesting vizualizations. 

It do not contain a major number of duplicates or missing values on important column, so the cleaning / pre-processing part will not be very difficult to tackle.

### Cleaning and Preprocessing

In [ ]:
clean_calls_data = calls_data.copy()

In [ ]:
# Drop duplicates in the description column
clean_calls_data = calls_data.drop_duplicates("desc")

# Drop the NA's
clean_calls_data = clean_calls_data.dropna()

# Create a new group column for each main category (EMS / Traffic / Fire) of the title column 
clean_calls_data['group'] = clean_calls_data['title'].apply(lambda x: x.split(':')[0])

# Create a year column for time series plots
clean_calls_data["year"] = clean_calls_data['timeStamp'].dt.year

clean_calls_data.head()

After checking the zip codes and the map, there are some outliers to the data set (c.f. zip codes of Montgomery County, Pennsylvania : https://www.ciclt.net/sn/clt/capitolimpact/gw_ziplist.aspx?FIPS=42091) : it does not contain zip codes that are starting with the number 17.

It could be that some calls are outside of the county and the emergency services went outside of the county.

We decided to focus more on the county and less of these "outliers".

Let us filter these out.

In [ ]:
clean_calls_data['zip_initial'] = clean_calls_data['zip'].str[:2]

clean_calls_data['zip_initial'] = pd.to_numeric(clean_calls_data['zip_initial'])

clean_calls_data = clean_calls_data.query('zip_initial >= 18')

#### Time series

In [ ]:
n_calls_per_year = clean_calls_data.groupby('year')['e'].agg(['count'])

n_calls_per_year = pd.DataFrame(n_calls_per_year).reset_index()

n_calls_per_year

In [ ]:
sns.lineplot(data = n_calls_per_year, x = 'year', y = 'count')
plt.title('Number of calls per year')

It would make sense to start the data from 2016 on, since it does not contain loads of data. 

For the decline in 2020, we could remove some of the months since the slope seems less important than the one from 2015-2016, but let us first analyze those years.

In [ ]:
clean_calls_data = clean_calls_data[(clean_calls_data.timeStamp >= "2016-01-01 00:00:00")]

It seems that the 2020 year have much less 911 calls than the other years. Could that be because another event (like COVID for example) or is it because the data set is incomplete ? (or both) 

Let us take a deeper look at the 2019 and 2020 year, specifically.

In [ ]:
calls_19 = clean_calls_data.query('year >= 2019 ')

calls_19['month_year'] = calls_19['timeStamp'].apply(lambda x: x.strftime('%m-%Y'))

# Create a pivot table to count the calls per week
calls_pivot = pd.pivot_table(calls_19, values='e', index=['timeStamp'], columns=['group'], aggfunc=np.sum)

# Creating counts per group per week
calls_pivot = calls_pivot.resample('W').agg(np.sum).reset_index()
calls_pivot.head()

In [ ]:
fig, ax = plt.subplots()

ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left() 
plt.xticks(fontsize = 12) 

ax.plot_date(calls_pivot['timeStamp'], calls_pivot['EMS'],'k', label = 'EMS')
ax.plot_date(calls_pivot['timeStamp'], calls_pivot['Traffic'],'b', label = 'Traffic')
ax.plot_date(calls_pivot['timeStamp'], calls_pivot['Fire'],'r', label = 'Fire')


ax.set_title("EMS, Fire and Traffic 911 calls for the 2019 - 2020 years")
fig.autofmt_xdate()
plt.legend(loc = 'upper left')
plt.show()


We can observe two things: 

- The data set stops at august 2020, not as the other years as they are complete. The 2020 year has 4 months of calls missing compared to the other years.

- There is a decrease in the calls around march-april 2020 ; it could be correlated with the COVID-19 crisis but further researches to support that hypothesis must be done.

We should remove the end months of 2020, since after the decrease there seems to be a significant increase in calls.

In [ ]:
calls_pivot_20 = calls_pivot[(calls_pivot.timeStamp >= "2020-01-01 00:00:00")]

In [ ]:
fig, ax = plt.subplots()

ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left() 
plt.xticks(fontsize=12) 

ax.plot_date(calls_pivot_20['timeStamp'], calls_pivot_20['EMS'], 'k', label = 'EMS')
ax.plot_date(calls_pivot_20['timeStamp'], calls_pivot_20['Traffic'], 'b', label = 'Traffic')
ax.plot_date(calls_pivot_20['timeStamp'], calls_pivot_20['Fire'], 'r', label = 'Fire')


ax.set_title("EMS, Fire and Traffic 911 calls for the 2020 year")
fig.autofmt_xdate()
plt.legend(loc = 'upper left')
plt.show()

It seems reasonable to cut July and August of the time frame for the 2020 year, which contains much less data than the other months.

In [ ]:
july_august = calls_pivot_20[(calls_pivot_20.timeStamp > "2020-06-28 00:00:00")]

july_august

In [ ]:
clean_calls_data = clean_calls_data[(clean_calls_data.timeStamp <= "2020-06-28 00:00:00")]

In [ ]:
clean_calls_data['month_year'] = clean_calls_data['timeStamp'].apply(lambda x: x.strftime('%m-%Y'))

# Create a pivot table to count the calls per week
clean_calls_pivot = pd.pivot_table(clean_calls_data, values='e', index=['timeStamp'], columns=['group'], aggfunc=np.sum)

# Creating counts per group per week
clean_calls_pivot = clean_calls_pivot.resample('W').agg(np.sum).reset_index()
clean_calls_pivot.head()

In [ ]:
fig, ax = plt.subplots()

ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left() 
plt.xticks(fontsize = 12) 

ax.plot_date(clean_calls_pivot['timeStamp'], clean_calls_pivot['EMS'],'k', label = 'EMS')
ax.plot_date(clean_calls_pivot['timeStamp'], clean_calls_pivot['Traffic'],'b', label = 'Traffic')
ax.plot_date(clean_calls_pivot['timeStamp'], clean_calls_pivot['Fire'],'r', label = 'Fire')


ax.set_title("EMS, Fire and Traffic 911 calls from 2016 - 2020")
fig.autofmt_xdate()
plt.legend(loc = 'upper left')
plt.show()

### First look at the revenues data

In [ ]:
revenues_data.info()

We miss 9 revenue values. Except that, we have access to the median revenue of each zipcode present in the calls dataset.

### Cleaning and preprocessing

To enable further work, we will approximate the missing revenues as the average of all the other revenues. This approach is definitely questionable, but it just a first approximation.

In [ ]:
average_med_revenue = revenues_data[revenues_data["med_revenue"].notnull()]["med_revenue"].mean()
clean_revenues_data = revenues_data
clean_revenues_data["med_revenue"] = revenues_data["med_revenue"].fillna(int(average_med_revenue))

## Exploratory Data Analysis

Let us begin with vizualizing just simple counts of the values in our data set.

In [ ]:
clean_calls_data["title"].value_counts()[:20].plot(kind = "barh")
plt.title('20 most common 911 calls in Montgomery County, PA for the 2016-2020 period')
plt.xlabel('Number of 911 calls')


Traffic problems have the most calls over the years, followed by fire alarm accidents and fall victim emergencies.

Vehicle accidents surpass the others by far.

In [ ]:
clean_calls_data["group"].value_counts()

We can see that EMS (Emergency medical services) are the highest number of the calls made in the county, followed by traffic and fire.

Although, it seems that it is because Traffic only have few titles compared to the EMS one, that is categorize into more several titles.

#### Calls per time of the day

Let us plot a heatmap of the most common call type and see how it evolve depending on the hour of the day.

In [ ]:
vehicle_accidents = clean_calls_data[(clean_calls_data.title.str.match(r'EMS:.*VEHICLE ACCIDENT.*') | clean_calls_data.title.str.match(r'Traffic:.*VEHICLE ACCIDENT.*'))]

vehicle_accidents['Month'] = vehicle_accidents['timeStamp'].apply(lambda x: x.strftime('%m %B'))
vehicle_accidents['Hour'] = vehicle_accidents['timeStamp'].apply(lambda x: x.strftime('%H'))

pivot_table_accidents = pd.pivot_table(vehicle_accidents, values='e', index=['Month'] , columns=['Hour'], aggfunc=np.sum)

pivot_table_accidents.head()

In [ ]:
cmap = sns.cubehelix_palette(light = 2, as_cmap = True)

ax = sns.heatmap(pivot_table_accidents, cmap = cmap)

ax.set_title('Vehicle Accidents during the period 2016-2020, over all townships in Montgomery County, PA')

We can observe that most of the vehicle accidents are during november / december / january, especially at 5 pm.

It could be because it is the end of the day so people are more tired and focused than the beginning of the day; it corresponds fairly to the end of the working time, people could be tired and want to go home rather quickly.

It could also be because during winter there is less visibilty on the road or more severe weather than during the spring or summer.

Sun goes down at approximatively 4.30 pm during the winter, in december (c.f. https://www.timeanddate.com/sun/@5201756?month=12&year=2019), which could explain more accident.

#### Calls per regions

In [ ]:
regions = clean_calls_data.copy()

sorted_regions = regions.groupby('zip')['e'].agg(['count']).sort_values('count', ascending = False)

sorted_regions = pd.DataFrame(sorted_regions).reset_index()

sorted_regions.head(10)

It clearly seems that there are "hotspots" for calls in the county.

In [ ]:
with urlopen('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/pa_pennsylvania_zip_codes_geo.min.json') as response:
    zipcodes = json.load(response)

In [ ]:
fig = px.choropleth(
    data_frame = sorted_regions, 
    geojson = zipcodes,
    locations = 'zip', 
    color = 'count',
    featureidkey = "properties.ZCTA5CE10",
    labels = {"count": "number of calls"}
)
fig.update_layout(margin = {"r": 0, "t": 0, "l": 0, "b": 0})
fig.update_geos(fitbounds = "locations", visible = True)

In [ ]:
sorted_regions_grouped = clean_calls_data.groupby(['zip', 'group'])['e'].agg(['count']).sort_values('count', ascending=False)

sorted_regions_grouped = pd.DataFrame(sorted_regions_grouped).reset_index()

sorted_regions_grouped.head(10)

### Revenue per zipcode

In [ ]:
fig = px.choropleth(
    data_frame = clean_revenues_data, 
    geojson = zipcodes,
    locations = 'zipcode', 
    color = 'med_revenue',
    featureidkey = "properties.ZCTA5CE10",
    labels = {"med_revenue": "median revenue($)"}
)
fig.update_layout(margin = {"r":0, "t":0, "l":0, "b":0})
fig.update_geos(fitbounds = "locations", visible = True)